In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv(f"../data/bbc_text_cls.csv")
print(df['labels'].value_counts())
df = df[df['labels'] == 'business']
print(df['labels'].value_counts())

labels
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64
labels
business    510
Name: count, dtype: int64


In [4]:
df.drop(columns='labels', inplace=True)
df.head()

,text
0,Ad sales boost Time Warner profit\n\nQuarterly...
1,Dollar gains on Greenspan speech\n\nThe dollar...
2,Yukos unit buyer faces loan claim\n\nThe owner...
3,High fuel prices hit BA's profits\n\nBritish A...
4,Pernod takeover talk lifts Domecq\n\nShares in...


In [8]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'\n', ' ', regex=True)
df['text'] = df['text'].str.replace(r'[^\w\s]', '', regex=True)
df['text'] = df['text'].str.replace(r'[\d]', '', regex=True)
df['text_list'] = df['text'].apply(lambda x: x.split())
df.head()

,text,text_list
0,ad sales boost time warner profit quarterly p...,"[ad, sales, boost, time, warner, profit, quart..."
1,dollar gains on greenspan speech the dollar h...,"[dollar, gains, on, greenspan, speech, the, do..."
2,yukos unit buyer faces loan claim the owners ...,"[yukos, unit, buyer, faces, loan, claim, the, ..."
3,high fuel prices hit bas profits british airw...,"[high, fuel, prices, hit, bas, profits, britis..."
4,pernod takeover talk lifts domecq shares in u...,"[pernod, takeover, talk, lifts, domecq, shares..."


In [9]:
#unique word list
word_set = set()

for text in df['text_list']:
    word_set.update(text)
idx2word = list(word_set)
idx2word.insert(0, '<UNKNOWN>')

word2idx = {word:i for i, word in enumerate(idx2word)}
print(len(word2idx))
print(list(word2idx.items())[0:5])

11963
[('<UNKNOWN>', 0), ('vendetta', 1), ('cao', 2), ('assistant', 3), ('bosphorus', 4)]


In [12]:
def word_numerizer(arr):
    return [word2idx.get(word, 0) for word in arr]

In [13]:
df['text_list_int'] = df['text_list'].apply(lambda x: word_numerizer(x))
df['text_list_int']

0      [8762, 3124, 10523, 4415, 10145, 7315, 3622, 7...
1      [7037, 111, 5734, 8444, 11250, 5377, 7037, 657...
2      [5476, 3791, 5091, 6304, 6110, 2202, 5377, 307...
3      [2511, 3683, 4871, 701, 10810, 7525, 1972, 119...
4      [8903, 8344, 7473, 898, 8193, 11510, 10165, 67...
                             ...                        
505    [10111, 8948, 2595, 11163, 646, 1523, 5377, 10...
506    [6713, 1254, 1580, 3913, 7897, 11768, 5377, 67...
507    [8477, 6612, 10599, 5734, 10111, 5377, 11538, ...
508    [4025, 5500, 7856, 1896, 5734, 6883, 9965, 598...
509    [2101, 5011, 3040, 8029, 3864, 6440, 5893, 155...
Name: text_list_int, Length: 510, dtype: object